# Install Packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten, Activation, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
import itertools
import shutil
import os
import random
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import datetime
from time import time
from my_mobilenet import myMobileNetV1

# Using GPU

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Variable

In [3]:
TRAIN_PATH = './data/seg_train/seg_train/'
VALID_PATH = './data/seg_test/seg_test/'
TEST_PATH = './data/seg_pred/'
IMAGE_SIZE = (224, 224, 3)
BATCH_SIZES = 64
EPOCHS = 80
switch = "my_mobilenet" #test model

# Data Preparetion

In [4]:
datagen = ImageDataGenerator()
train_batches = datagen.flow_from_directory(directory=TRAIN_PATH, color_mode="rgb",target_size=IMAGE_SIZE[:-1], class_mode="categorical", shuffle=True,batch_size=BATCH_SIZES)
valid_batches = datagen.flow_from_directory(directory=VALID_PATH, color_mode="rgb",target_size=IMAGE_SIZE[:-1], class_mode="categorical", shuffle=False,batch_size=BATCH_SIZES)
test_batches = datagen.flow_from_directory(directory=TEST_PATH,target_size=IMAGE_SIZE[:-1],color_mode="rgb",batch_size=BATCH_SIZES,class_mode=None,shuffle=False)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 7301 images belonging to 1 classes.


# Model Construct

In [5]:
if switch == "my_mobilenet":
    model = myMobileNetV1(input_shape = IMAGE_SIZE, num_classes = 6 ) 
else:
    if switch == "mobilenet":
        md = keras.applications.MobileNet(input_shape = IMAGE_SIZE, weights=None)
    elif switch == "mobilenetv2":
        md = keras.applications.MobileNetV2(input_shape = IMAGE_SIZE, weights=None)
    elif switch == "ResNet50":
        md = keras.applications.ResNet50(input_shape = IMAGE_SIZE, weights=None)
    x = Flatten()(md.output)
    prediction = Dense(6, activation='softmax')(x)
    model = Model(inputs=md.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 32)      0     

In [6]:
optimizer = Adam(learning_rate = 1e-3)
model.compile(
    loss='categorical_crossentropy',
    optimizer= optimizer,
    metrics=['accuracy']
)
stop_callback = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience = 4, verbose=1, min_lr = 1e-6)
t1 = time()
history = model.fit(train_batches, validation_data = valid_batches, epochs= EPOCHS, steps_per_epoch=len(train_batches), validation_steps=len(valid_batches), callbacks = [stop_callback,reduce_lr])
t2 = time()
print("execution time: ", t2 - t1)

Epoch 1/80
220/220 [==============================] - 62s 229ms/step - loss: 1.1983 - accuracy: 0.5214 - val_loss: 4.5750 - val_accuracy: 0.1843: 1.3693 - accura - ETA: 20s - loss: 1.3476 - a -
Epoch 2/80
220/220 [==============================] - 49s 223ms/step - loss: 0.6619 - accuracy: 0.7550 - val_loss: 4.3249 - val_accuracy: 0.1750
Epoch 3/80
220/220 [==============================] - 50s 225ms/step - loss: 0.5207 - accuracy: 0.8120 - val_loss: 2.0833 - val_accuracy: 0.3910
Epoch 4/80
220/220 [==============================] - 50s 228ms/step - loss: 0.4584 - accuracy: 0.8368 - val_loss: 0.8729 - val_accuracy: 0.7323
Epoch 5/80
220/220 [==============================] - 50s 225ms/step - loss: 0.3943 - accuracy: 0.8574 - val_loss: 0.7176 - val_accuracy: 0.7690
Epoch 6/80
220/220 [==============================] - 50s 227ms/step - loss: 0.3475 - accuracy: 0.8777 - val_loss: 0.7068 - val_accuracy: 0.7780
Epoch 7/80
220/220 [==============================] - 50s 228ms/step - loss: 0.30

In [7]:
scoreSeg = model.evaluate(valid_batches)
print("Test Data Accuracy = ",scoreSeg[1])

47/47 [==============================] - 3s 63ms/step - loss: 0.4595 - accuracy: 0.8700
Test Data Accuracy =  0.8700000047683716


# Result

In [9]:
from sklearn.metrics import classification_report
test_labels = valid_batches.classes 
predictions = model.predict(valid_batches, verbose=1)
y_pred = np.argmax(predictions, axis=-1)
print(classification_report(test_labels, y_pred, target_names = valid_batches.class_indices))

47/47 [==============================] - 3s 63ms/step
              precision    recall  f1-score   support

   buildings       0.88      0.84      0.86       437
      forest       0.96      0.97      0.96       474
     glacier       0.83      0.79      0.81       553
    mountain       0.83      0.83      0.83       525
         sea       0.85      0.90      0.87       510
      street       0.89      0.90      0.89       501

    accuracy                           0.87      3000
   macro avg       0.87      0.87      0.87      3000
weighted avg       0.87      0.87      0.87      3000



In [10]:
print(tf.__version__)
print(tf.keras.__version__)
#print(scipy.__version__)

2.4.0
2.4.0
